In [1]:
#!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [2]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from googleapiclient.discovery import build
import re
from datetime import datetime

In [5]:
file = open("API_GOOGLE.txt", "r")
api_key = file.read()
file.close()

In [6]:
channel_id = ['UChBEbMKI1eCcejTtmI32UEw','UCfE5Cz44GlZVyoaYTHJbuZw','UCMyOj6fhvKFMjxUCp3b_3gA']
youtube = build('youtube', 'v3', developerKey= api_key)

In [7]:
def get_channel_stats(youtube, channel_id):
    request = youtube.channels().list(
    part="snippet,contentDetails,statistics",
        id = channel_id)
    response = request.execute()
    return response['items']

In [8]:
channel_stats = get_channel_stats(youtube, channel_id)

In [9]:
playlist_id = channel_stats[0]['contentDetails']['relatedPlaylists']['uploads']

In [10]:
def get_video_list(youtube, upload_id):
    video_list = []
    request = youtube.playlistItems().list(
    part="snippet,contentDetails",
    playlistId = upload_id,
    maxResults=50
    )
    
    next_page = True
    
    while next_page:
        response = request.execute()
        data = response['items']
        
        for video in data:
            video_id = video['contentDetails']['videoId']
            if video_id not in video_list:
                video_list.append(video_id)
                
        if 'nextPageToken' in response.keys():
            next_page = True
            
            request = youtube.playlistItems().list(
                part="snippet,contentDetails",
                playlistId = upload_id,
                maxResults=50,
                pageToken=response['nextPageToken']
            )
        else:
            next_page = False
            
    return video_list

In [11]:
def get_video_details(youtube, video_list):
    stats_list = []
    
    for i in range(0, len(video_list), 50):
        request = youtube.videos().list(
            part="snippet,contentDetails, statistics",
            id=video_list[i:i+50]
        )
        
        data = request.execute()
        
        for video in data['items']:
            title = video['snippet']['title']
            published = video['snippet']['publishedAt']
            description = video['snippet']['description']
            tag_count = len(video['snippet'].get('tags',[]))
            tag = "No tags" if tag_count == 0 else "Have tags"
            view_count = video['statistics'].get('viewCount',0)
            like_count = video['statistics'].get('likeCount',0)
            dislike_count = video['statistics'].get('dislikeCount',0)
            comment_count = video['statistics'].get('commentCount',0)
            
            stats_dictionary = dict(title = title, 
                                    published = published, 
                                    description = description,
                                    tag = tag,
                                    view_count = view_count,
                                    like_count = like_count,
                                    comment_count = comment_count
                                   )
            stats_list.append(stats_dictionary)
    return stats_list

In [12]:
#channel_stats[0]['statistics']

In [13]:
#video_list = get_video_list(youtube, playlist_id)

In [14]:
#video_data = get_video_details(youtube, video_list)

In [15]:
#len(video_data)

In [16]:
#df = pd.DataFrame(video_data)

In [17]:
def get_all_video_data_for_channels(youtube, channel_ids):
    all_video_data = []

    
    for ch_id in channel_ids:
        channel_stats = get_channel_stats(youtube, [ch_id]) 

        if not channel_stats:  
            continue
        
    
        channel_name = channel_stats[0]['snippet']['title']

        playlist_id = channel_stats[0]['contentDetails']['relatedPlaylists']['uploads']
        video_list = get_video_list(youtube, playlist_id)
        video_data = get_video_details(youtube, video_list, channel_name)  # Passing channel name
        
        all_video_data.extend(video_data)

    return all_video_data

def get_video_details(youtube, video_list, channel_name):
    stats_list = []
    
    for i in range(0, len(video_list), 50):
        request = youtube.videos().list(
            part="snippet,contentDetails, statistics",
            id=video_list[i:i+50]
        )
        
        data = request.execute()
        
        for video in data['items']:
            title = video['snippet']['title']
            published = video['snippet']['publishedAt']
            description = video['snippet']['description']
            length = video['contentDetails']['duration']
            tag_count = len(video['snippet'].get('tags',[]))
            tag = "No tags" if tag_count == 0 else "Have tags"
            view_count = video['statistics'].get('viewCount',0)
            like_count = video['statistics'].get('likeCount',0)
            comment_count = video['statistics'].get('commentCount',0)
            

            stats_dictionary = dict(
                channel_name = channel_name, 
                title = title, 
                published = published, 
                description = description,
                length = length,
                tag = tag,
                view_count = view_count,
                like_count = like_count,
                comment_count = comment_count 
            )
            stats_list.append(stats_dictionary)
    return stats_list

video_data = get_all_video_data_for_channels(youtube, channel_id)
df = pd.DataFrame(video_data)


In [18]:
df

,channel_name,title,published,description,length,tag,view_count,like_count,comment_count
0,Joshua Weissman,Creamy Butter Chicken Sandwich,2023-10-26T17:00:06Z,,PT33S,No tags,444031,37107,307
1,Joshua Weissman,I Tried Food From Every State In America,2023-10-17T14:30:12Z,Get the NEW COOKBOOK: https://bit.ly/TextureOv...,PT33M53S,Have tags,3583404,114920,21522
2,Joshua Weissman,The Greatest Chicken Nuggets,2023-10-15T22:00:30Z,,PT24S,No tags,1489893,125668,534
3,Joshua Weissman,Meals So Easy A College Student Could Make It,2023-10-13T14:30:09Z,"Cheap and easy meals that everyone can make, w...",PT21M56S,Have tags,1890863,79081,1917
4,Joshua Weissman,I Made McDonald's French Fries Better,2023-10-08T14:30:06Z,If we can do this then did we make the best fr...,PT10M17S,Have tags,1193013,42536,2031
...,...,...,...,...,...,...,...,...,...
1770,Nick DiGiovanni,How To Cook Arctic Char,2020-10-13T01:56:10Z,"The perfect combination of salmon and trout, s...",PT4M55S,Have tags,429848,13063,378
1771,Nick DiGiovanni,How To Cook Salmon,2020-09-29T00:12:38Z,This is salmon made easy. Follow my 80-20 rule...,PT2M31S,Have tags,443527,14626,282
1772,Nick DiGiovanni,Beef Wellington,2020-09-14T18:41:27Z,Beef Wellington doesn't have to be scary. \n\n...,PT8M5S,Have tags,828906,27456,1114
1773,Nick DiGiovanni,Japanese A5 Wagyu Beef,2020-08-31T21:42:22Z,You can almost eat this steak with a spoon. \n...,PT10M29S,Have tags,792778,20369,592


In [19]:
#Cleaning data
#Check ft in title, to see if collabaration plays a role in views
df['collaboration'] = df['title'].str.contains('ft|Ft', case=False)

In [20]:
#Change length to minutes
def convert_to_minutes(iso_duration):
    minutes_match = re.search(r'(\d+)M', iso_duration)
    seconds_match = re.search(r'(\d+)S', iso_duration)
    
    minutes = int(minutes_match.group(1)) if minutes_match else 0
    seconds = int(seconds_match.group(1)) if seconds_match else 0

    return minutes + seconds/60


df['duration_in_minutes'] = df['length'].apply(convert_to_minutes)

In [22]:
#Get day of publication
df['published'] = df['published'].apply(lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M:%SZ"))

def get_weekday(date_string):
    return date_string.strftime("%A").lower() 

df['day_published'] = df['published'].apply(get_weekday)
df

,channel_name,title,published,description,length,tag,view_count,like_count,comment_count,collaboration,duration_in_minutes,day_published
0,Joshua Weissman,Creamy Butter Chicken Sandwich,2023-10-26 17:00:06,,PT33S,No tags,444031,37107,307,False,0.550000,thursday
1,Joshua Weissman,I Tried Food From Every State In America,2023-10-17 14:30:12,Get the NEW COOKBOOK: https://bit.ly/TextureOv...,PT33M53S,Have tags,3583404,114920,21522,False,33.883333,tuesday
2,Joshua Weissman,The Greatest Chicken Nuggets,2023-10-15 22:00:30,,PT24S,No tags,1489893,125668,534,False,0.400000,sunday
3,Joshua Weissman,Meals So Easy A College Student Could Make It,2023-10-13 14:30:09,"Cheap and easy meals that everyone can make, w...",PT21M56S,Have tags,1890863,79081,1917,False,21.933333,friday
4,Joshua Weissman,I Made McDonald's French Fries Better,2023-10-08 14:30:06,If we can do this then did we make the best fr...,PT10M17S,Have tags,1193013,42536,2031,False,10.283333,sunday
...,...,...,...,...,...,...,...,...,...,...,...,...
1770,Nick DiGiovanni,How To Cook Arctic Char,2020-10-13 01:56:10,"The perfect combination of salmon and trout, s...",PT4M55S,Have tags,429848,13063,378,False,4.916667,tuesday
1771,Nick DiGiovanni,How To Cook Salmon,2020-09-29 00:12:38,This is salmon made easy. Follow my 80-20 rule...,PT2M31S,Have tags,443527,14626,282,False,2.516667,tuesday
1772,Nick DiGiovanni,Beef Wellington,2020-09-14 18:41:27,Beef Wellington doesn't have to be scary. \n\n...,PT8M5S,Have tags,828906,27456,1114,False,8.083333,monday
1773,Nick DiGiovanni,Japanese A5 Wagyu Beef,2020-08-31 21:42:22,You can almost eat this steak with a spoon. \n...,PT10M29S,Have tags,792778,20369,592,False,10.483333,monday
